Цель - собрать текстовые данные и среднее количество просмотров с новостного сайта fontanka.ru.

# Парсим новости с fontanka.ru

In [1]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек 
import time          # Библиотека для времени
%matplotlib inline
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from tqdm import tqdm_notebook
from sklearn.metrics import *
import warnings
warnings.filterwarnings("ignore")

## Делаем пробный запрос

In [2]:
#page_link = 'https://www.fontanka.ru/2022/11/19/71829761/'
#page_link = 'https://www.fontanka.ru/2022/11/19/71830307/'
page_link = 'https://www.fontanka.ru/2022/11/18/71828858/'

In [3]:
response = requests.get(page_link)
response

<Response [200]>

Т.к. response = 200, не нужно использовать useragent.

html = response.content

In [5]:
len(html)

798760

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser') # В опции также можно указать lxml, 
                                         # если предварительно установить одноименный пакет

In [7]:
type(soup)

bs4.BeautifulSoup

In [8]:
print(soup.prettify()[:200])

<!DOCTYPE html>
<html lang="ru">
 <head>
  <meta charset="utf-8"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="width=device-width,initial-scale=1,minimum-scale=1,maximum-sca


Вытащим количество просмотров.

In [9]:
views = soup.find(attrs={'class':'HZacv primaryOverlineMobile'})
views = views.text
views = pd.to_numeric(views)
views

6316

Вытащим дату публикации объявления и найдем сколько дней назад опубликовано объявление.

In [10]:
from datetime import datetime
date = soup.find(attrs={'itemprop':'datePublished'})
date = date.get('content')
date = pd.to_datetime(date, format='%Y-%m-%dT%H:%M:%S')
date=date.to_pydatetime()
#вытащим день, т.к. все статьи за ноябрь 2022, то месяц не нужен.
day = date.strftime("%d")
day = pd.to_numeric(day)
#найдем текущий день
current_day =datetime.now().strftime("%d")
current_day = pd.to_numeric(current_day)
#посчитаем сколько дней опубликовано объявление
days_ago = current_day - day
days_ago

3

Найдем количество просмотров в день.

In [11]:
views_in_day = views/days_ago
views_in_day

2105.3333333333335

Найдем заголовок.

In [12]:
title = soup.find(attrs={'class':'primaryH2HeadlineTablet'})
title.text

'Смольный изменил прогноз по количеству туристов. Иностранцев и круизников пока не ждут'

Найдем весь текст.

In [13]:
#Найдем все текстовые части статьи. 
body = soup.find_all(attrs={'class':'G7al ETdx ETal'})
#цикл для сложения частей текстов из списка 
for i in range(len(body)):
    text=body[0].text+body[i].text
    i += 1
text

'Власти Петербурга скорректировали прогнозы по развитию сферы туризма. В 2022 году Северную столицу посетят 6,3 млн человек, говорится в проекте изменений госпрограммы «Развитие сферы туризма в Санкт-Петербурге», опубликованном 18 ноября на портале антикоррупционной экспертизы.В прежней версии программы содержится прогноз турпотока на уровне 5,1 млн человек. Однако ранее губернатор Александр Беглов сообщал, что по итогам 2022 года город может принять более 7 млн туристов. Эта планка будет преодолена, если удастся возобновить турпоток из дружественных стран.Прогноз восстановления круизного турпотока на ближайшие годы исчез. Но в 2025 году Смольный рассчитывает принять 910,9 тыс. гостей. Ранее такого показателя ожидали в 2023 году.'

## Создадим функцию для сбора информации по списку ссылок

In [14]:
#Cоздадим список ссылок
link_list =['https://www.fontanka.ru/2022/11/19/71829761/', 
            'https://www.fontanka.ru/2022/11/19/71830307/',
           'https://www.fontanka.ru/2022/11/18/71828858/']

In [15]:
def getData(list):
    """
        Запрашивает данные по странице, возвращает обработанный словарь с данными
            
    """
    # запрашиваем данные по ссылке
    response = requests.get(list)
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    #Вытащим количество просмотров
    views = soup.find(attrs={'class':'HZacv primaryOverlineMobile'})
    views = views.text
    views = pd.to_numeric(views)
    
    # Вытащим дату публикации объявления 
    #и найдем сколько дней назад опубликовано объявление.
    date = soup.find(attrs={'itemprop':'datePublished'})
    date = date.get('content')
    date = pd.to_datetime(date, format='%Y-%m-%dT%H:%M:%S')
    date=date.to_pydatetime()
    #вытащим день, т.к. все статьи за ноябрь 2022, то месяц не нужен.
    day = date.strftime("%d")
    day = pd.to_numeric(day)
    #найдем текущий день
    current_day =datetime.now().strftime("%d")
    current_day = pd.to_numeric(current_day)
    #посчитаем сколько дней опубликовано объявление
    days_ago = current_day - day
    #найдем количество просмотров в день
    views_in_day = round(views/days_ago, 0)

    # найдем загловок
    title = soup.find(attrs={'class':'primaryH2HeadlineTablet'})
    title = title.text

    #Найдем все текстовые части статьи. 
    body = soup.find_all(attrs={'class':'G7al ETdx ETal'})
    #цикл для сложения частей текстов из списка 
    for i in range(len(body)):
        text=body[0].text+body[i].text
        i += 1
    text

    # составляем словарь, в котором будут хранится все полученные и обработанные данные
    data_row = {"title":title, "views_in_day":views_in_day, 
                "text":text}
    return data_row

In [16]:
#создадим таблицу с колонками
final_df = pd.DataFrame(columns=['title', 'views_in_day', 'text'])

In [17]:
#Проходим по списку ссылок циклом создавая датафрейм
for link_list in tqdm_notebook(link_list):
    try: 
        data_row = getData(link_list)
        final_df = final_df.append(data_row, ignore_index=True)
        time.sleep(0.3)
    except:
        continue

  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
#удалим возможные пропуски и дубликаты
final_df = final_df.drop_duplicates().dropna(axis = 1)

In [19]:
final_df.shape

(3, 3)

In [20]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
final_df

,title,views_in_day,text
0,"«Фонтанка» показывает, что так громко стреляет в центре Петербурга",55695.0,"Читатели «Фонтанки» присылают сообщения о звуках громкой стрельбы, которые слышат в разных районах городах. В субботу, 19 ноября, отмечают День ракетных войск и артиллерии — у Петропавловской крепости проходит мероприятие с участием высоких гостей.Супервидовая камера «Фонтанки» засняла на видео происходящее там.На 12:00 был запланирован салют на Петропавловской крепости. Как сообщают в военном ведомстве, звуки выстрелов могут быть слышны с 10 до 14 часов в субботу."
1,"«Вид на непонятный столб огня, сильнейший гул». Около Мурино огромный пожар",89086.0,"Около Мурино, как сообщают читатели «Фонтанки», заметили возгорание. Публикуем фотографии и видео.«Вид на непонятный столб огня, видно, в Мурино. Сильнейший гул», — сообщает очевидец.Губернатор Ленобласти Александр Дрозденко сообщил, что пожар возник из-за взрыва на газопроводе между Бернгардовкой и Ковалево во Всеволожском районе. Подробнее тут."
2,Смольный изменил прогноз по количеству туристов. Иностранцев и круизников пока не ждут,2108.0,"Власти Петербурга скорректировали прогнозы по развитию сферы туризма. В 2022 году Северную столицу посетят 6,3 млн человек, говорится в проекте изменений госпрограммы «Развитие сферы туризма в Санкт-Петербурге», опубликованном 18 ноября на портале антикоррупционной экспертизы.В прежней версии программы содержится прогноз турпотока на уровне 5,1 млн человек. Однако ранее губернатор Александр Беглов сообщал, что по итогам 2022 года город может принять более 7 млн туристов. Эта планка будет преодолена, если удастся возобновить турпоток из дружественных стран.Прогноз восстановления круизного турпотока на ближайшие годы исчез. Но в 2025 году Смольный рассчитывает принять 910,9 тыс. гостей. Ранее такого показателя ожидали в 2023 году."


**Вывод:** Получены название новости, количество просмотров каждой новости в день и собраны текстовые поля для каждой новости (сбор полей был необходим, т.к. зачастую они разделены изображениями или видео).